In [7]:
import sys
sys.path.append('../')
import numpy as np
import ekkono.primer as primer
import pandas as pd
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from scripts import helper_functions
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
## Torch libraries
import torch
import torch.nn as nn
import torch.optim as optim


class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=32, num_layers=1, output_size=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # take last time step output
        return out

In [8]:
## Train Data

test_dynamic_dataset = primer.DynamicDataset([])
df = primer.ArffReader.read_file('../data/long_train_track_orginal/long_train_track_with_compass_and_light.arff')
data = pd.DataFrame(df)
data.columns = ['compass', 'light']
# Convert both columns to numeric (safe even if already numeric)
data['compass'] = pd.to_numeric(data['compass'], errors='coerce')
timestamps = np.arange(len(data['compass'])) * 0.1
data['turn_speed'] = helper_functions.compute_turn_speed(data['compass'],timestamps=timestamps)
data['turn_speed'] = helper_functions.moving_average(data['turn_speed'],window_size=100)
data['light'] = pd.to_numeric(data['light'], errors='coerce')
data['turn_speed-30'] = data['turn_speed'].shift(-30)

In [9]:
def create_sequence_dataset(ts_data, seq_length, horizon):
    """
    Prepare input sequences and targets for time series forecasting.

    Args:
        ts_data (array-like): 1D array of time series data (already scaled).
        seq_length (int): Number of consecutive points in each input sequence.
        horizon (int): Forecasting horizon (steps ahead).

    Returns:
        X_tensor (torch.Tensor): Shape (num_samples, seq_length, 1)
        y_tensor (torch.Tensor): Shape (num_samples, 1)
    """
    X = []
    y = []
    max_i = len(ts_data) - seq_length - horizon + 1
    for i in range(max_i):
        X.append(ts_data[i : i + seq_length])
        y.append(ts_data[i + seq_length + horizon - 1])
    X = np.array(X)
    y = np.array(y)

    X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)  # (batch, seq, feature=1)
    y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(-1)  # (batch, 1)
    return X_tensor, y_tensor


In [10]:
# Seed for reproducibility
torch.manual_seed(0)
np.random.seed(0)
ts_data = data['turn_speed'].to_numpy()
split_idx = int(0.8 * len(ts_data))

train = ts_data[:split_idx]
test = ts_data[split_idx:]


scaler = MinMaxScaler()

ts_data_train_scaled = scaler.fit_transform(train.reshape(-1,1)).flatten()
ts_data_test_scaled = scaler.fit_transform(test.reshape(-1,1)).flatten()

seq_length = 10
horizon = 10

x_train_tensor,y_train_tensor = create_sequence_dataset(ts_data_train_scaled,seq_length=seq_length,horizon=horizon)
x_test_tensor,y_test_tensor = create_sequence_dataset(ts_data_test_scaled,seq_length=seq_length,horizon=horizon)


model = LSTMModel()

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)
epochs = 50


# Training loop
for epoch in range(epochs):
    
    model.train()
    optimizer.zero_grad()
    outputs = model(x_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# Example predictions on training data

model.eval()
with torch.no_grad():
    predictions = model(x_test_tensor).squeeze().numpy()

print("\nSample predictions vs actual:")
for i in range(5):
    print(f"Actual: {y_test_tensor.squeeze().numpy()[i]:.4f}  Predicted: {predictions[i]:.4f}")


Epoch [10/50], Loss: 0.0740
Epoch [20/50], Loss: 0.0542
Epoch [30/50], Loss: 0.0458
Epoch [40/50], Loss: 0.0433
Epoch [50/50], Loss: 0.0410

Sample predictions vs actual:
Actual: 0.8234  Predicted: 0.5446
Actual: 0.8996  Predicted: 0.5636
Actual: 0.9706  Predicted: 0.5537
Actual: 0.9734  Predicted: 0.5712
Actual: 0.9680  Predicted: 0.5616


In [11]:

import plotly.graph_objs as go
import plotly.io as pio

predictions_original = scaler.inverse_transform(predictions.reshape(-1,1)).flatten()
y_original = scaler.inverse_transform(y_test_tensor.numpy().reshape(-1,1)).flatten()
# Plot actual vs predicted values for the entire dataset


# Example data; replace with your own
# y_original = ...
# predictions_original = ...

trace_actual = go.Scatter(
    y=y_original,
    mode='lines',
    name='Actual',
    line=dict(color='blue')
)

trace_pred = go.Scatter(
    y=predictions_original,
    mode='lines',
    name='Predicted',
    line=dict(color='red')
)

layout = go.Layout(
    title='Actual vs Predicted Time Series',
    xaxis=dict(title='Sample Index'),
    yaxis=dict(title='Turn Speed'),
    legend=dict(x=0, y=1),
    template='plotly_white'
)

fig = go.Figure(data=[trace_actual, trace_pred], layout=layout)
fig.show()


### LSTM Summary

Training is done using Pytorch framework on 80 percent of the data and the prediction is done on 20 percent of the data.

### Comaprison metrics

LSTM TFlite vs Ekkono 
 - Metrics - size of model
 - RMSE - error 
 - Computation speed
 - Memory of the Model

### Feasbility of LSTM/GRU/RNN in Edge devices
 - Comparison with Ekkono SDK error metrics 
   - 
   - Is this doing a better job in long sequence datasets ( Nederman, Siemens)
  
 - Device Side
   - Device compatibility 
     - Static and Dynamic memory availble in device (Static mem to store sdk and the functionalities, Dynamic to run the Computation)
  


## 